In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
import numpy as np
import os
import json
import csv
from tqdm import tqdm 

In [3]:
# 사전 데이터 불러오기
word_to_id = pd.read_csv("word_to_id_all.csv", encoding='utf-8')
id_to_word = pd.read_csv("id_to_word_all.csv", encoding='utf-8')

# 정수화된 문장 데이터 불러오기
standard_data = pd.read_csv("standard_padding_all.csv", encoding='utf-8')
dialect_data = pd.read_csv("dialect_padding_all.csv", encoding='utf-8')

In [8]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
# tensorflow 같은 딥러닝 프레임워크는 주로 numpt 배열을 입력으로 받음
# 때문에 values 속성 사용해 Pandas 데이터프레임을 Numpy 배열로 변환
input_sequences = dialect_data.values
output_sequences = standard_data.values

In [9]:
# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [6]:
# 모델 구성
# 임베딩 레이어: 단어를 밀집된 벡터로 변환
# 첫번째 LSTM 레이어: 데이터의 패턴을 학습
# RepactVector 레이어: 출력 시퀀스의 길이만큼 입력을 반복
# 두번째 LSTM 레이어: 첫번째 LSTM 레이어와 마찬가지로 패턴을 학습
# Dense 레이어: 단어 사전의 크기에 따라 출력 크기 설정하고, softmax 함수 사용해 확률 분포 출력
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(word_to_id)+ 1, 256, input_length=input_sequences.shape[1]),
    tf.keras.layers.LSTM(256),
    tf.keras.layers.RepeatVector(output_sequences.shape[1]),
    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.Dense(len(word_to_id) + 1, activation='softmax')
])

In [7]:
# 모델 compile
# 모델이 훈련 가능한 상태가 되도록 하는 것
# optimizer는 학습률을 조절해 최적화하는 방법이며 adam은 RMSProp과 Momentum 방식을 결합한 방식
# Loss function은 손실 값을 측정하는 방법이며 여기서는 훈련 데이터의 라벨 값이 정수이므로 sparse_categorical_crossentropy 사용
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [11]:
print(type(input_sequences))
print(type(output_sequences))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [12]:
print(len(input_sequences))
print(len(output_sequences))
print(len(word_to_id))

367954
367954
128707


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# ModelCheckpoint 콜백 정의
checkpoint_callback = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)


In [ ]:
# 모델 학습 시 fit() 메서드의 callbacks 인자에 콜백 전달
history = model.fit(input_sequences, np.expand_dims(output_sequences, -1), epochs=100, batch_size=64, validation_split=0.2, callbacks=[checkpoint_callback])